# Part 2 - Export a model


In this section of the lab you will export the trained model and learn how to use it in your application. 
Custom Vision Service allows classifiers to be exported to run offline. You can embed your exported classifier into an application and run it locally on a device for real-time classification.

Custom Vision Service supports the following exports:

- Tensorflow for Android.
- CoreML for iOS11.
- ONNX for Windows ML.
- A Windows or Linux container. The container includes a Tensorflow model and service code to use the Custom Vision Service API.

Custom Vision Service only exports compact domains. The models generated by compact domains are optimized for the constraints of real-time classification on low powered devices. Classifiers built with a compact domain may be slightly less accurate than a standard domain with the same amount of training data.



## Before you start
Install `wget` package that will be used later in the lab

In [ ]:
# Install Custom Vision Service SDK  in the current Jupyter kernel
import sys
!{sys.executable} -m pip install wget

Retrieve your training and prediction keys.

Create a `trainer` object using your keys

In [ ]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageUrlCreateEntry

ENDPOINT = "https://southcentralus.api.cognitive.microsoft.com"

# update keys
training_key = ''
prediction_key = ''

trainer = CustomVisionTrainingClient(training_key, endpoint=ENDPOINT)

## Retrain with a compact domain
If your classifier was not trained with the compact domain (which is the case for the classifier trained during the first stage of the lab) you need to retrain it.

When you first created your project, it was configured with the `General` domain. A domain represents a pre-trained deep neural network that will be used as a base of your custom image classifier. Each domain optimizes the classifier for specific types of images. `General` is optimized for a broad range of classification tasks. If none of the other domains (`Food`, `Landmarks`, `Retail`, `Adult`) are appropriate, or you are unsure of which domain to choose, select the `General` domain.

The compact domains are optimized for the constraints of real-time classification on low powered and mobile devices. The models generated by compact domains can be exported to run locally.

In [ ]:
# Print the list of available domains
for domain in trainer.get_domains():
 print(domain)

First, find the ID of the `General (compact)` domain.

In [ ]:
# Find the ID of General (compact) domain
domain_name = 'General (compact)'
modeltype = 'Classification'
domain_id = None
for domain in trainer.get_domains():
    if domain.name == domain_name:
        if domain.type == modeltype:
            domain_id = domain.id
            print("Found domain: {} {} type with ID: {}".format(domain_name,modeltype,domain_id))
            break

if domain_id == None:
    print("Could not find domain: {}".format(domain_name))

Next find your project's ID using your project's name 

In [ ]:
# Find you project's ID
# Make sure project id same as previous training notebook
project_name = 'AerialClassifier'

project_id = None
project = None
for prj in trainer.get_projects():
    if prj.name == project_name:
        project_id = prj.id
        project = prj
        print("Found project: {0} known as {1}".format(project_id, project_name))
        break
        
if project_id == None:
    print("Could not find your project")


And finally, change the domain of your project.

In [ ]:
# Change the project's domain
project.settings.domain_id = domain_id
project = trainer.update_project(project_id, project)

### Retrain

You will use the same helper function you used in Part 1 of the lab to retrain the model.

In [ ]:
import time

def train(training_key, project_id):
    trainer = CustomVisionTrainingClient(training_key, endpoint=ENDPOINT)
    print("Starting training...")
    try:
        iteration = trainer.train_project(project_id)
        while (iteration.status != "Completed"):
            time.sleep(5)
            iteration = trainer.get_iteration(project_id, iteration.id)
            print ("Training status: " + iteration.status)      
        # The iteration is now trained. Make it the default project endpoint
        print("Training completed")
        trainer.update_iteration(project_id, iteration.id, is_default=True)
    except:
        print("No need to retrain. Retrieving default iteration")
        for iteration in trainer.get_iterations(project_id):
            if iteration.is_default:
                break

    return iteration.id

In [ ]:
# Start training
iteration_id = train(training_key, project_id)

## Export the iteration
Exporting a model is a two-step process. First you must request an export. It is an asynchronous call. After you requested the export, you should periodically check the status of the request and when the export package is ready you can download it using the returned URI.




Request the export and wait for till it is fulfilled.

In [ ]:
print("Requesting export for Iteration ID: {0}".format(iteration_id))

platform = 'TensorFlow'
flavor = 'Linux'

export = trainer.export_iteration(project_id, iteration_id, platform, flavor)
while (export.status != 'Done'):
   print ("Export status: " + export.status)
   time.sleep(5)
   export = trainer.get_exports(project_id, iteration_id)[0]

print("Export package ready. Download URI: {}", export.download_uri)    

Now you can download the package.

In [ ]:
import wget
import os

download_filename = 'aerialclassifier.zip'

print("Downloading from: {0}".format(export.download_uri))
wget.download(export.download_uri, download_filename)

The package has been exported as a zip file.

In [ ]:
%%sh
unzip aerialclassifier.zip
ls -l 

It contains two files: `model.pb` and `labels.txt`. These files represent the trained model and the classification labels. The model has been exported in TensorFlow protocol buffers format. You will need TensorFlow runtime to execute model.

## Run TensorFlow model 

Install TensorFlow and other dependencies required to load and run your model.

In [ ]:
import sys
!{sys.executable} -m pip install tensorflow==1.7
!{sys.executable} -m pip install pillow
!{sys.executable} -m pip install numpy

You can now load the saved model to a TensorFlow computational graph.

In [ ]:
import tensorflow as tf
import os

filename = 'model.pb'
labels_filename = 'labels.txt'

graph_def = tf.GraphDef()
labels = []

# Import the TF graph
with tf.gfile.FastGFile(filename, 'rb') as f:
    graph_def.ParseFromString(f.read())
    tf.import_graph_def(graph_def, name='')

# Create a list of labels.
with open(labels_filename, 'rt') as lf:
    for l in lf:
        labels.append(l.strip())

### Prepare an image for prediction

There are a few steps for preparing the image so that it's the right shape for prediction. The exported model requires images to be BGR format of size (227, 227). Our testing images are RGB of size (224, 224) 


In [ ]:
from PIL import Image
import numpy as np

# Load from a file
imageFile = "test_images/developed-1.png"
image = Image.open(imageFile)
print(image
     )
# Resize 
image = image.resize((224, 224), resample = Image.BILINEAR)
print(image)

# Convert to numpy array - tensor
image_tensor = np.asarray(image)

# Convert RGB -> BGR 
r,g,b = np.array(image_tensor).T
image_tensor = np.array([b,g,r]).transpose()

print("Numpy array mode=BGR shape={}".format(image_tensor.shape))


### Predict an image
Once the image is prepared as a tensor of the required shape and format, we can send it through the model for a prediction.

In [ ]:
# These names are part of the model and cannot be changed.
output_layer = 'loss:0'
input_node = 'Placeholder:0'

with tf.Session() as sess:
    prob_tensor = sess.graph.get_tensor_by_name(output_layer)
    predictions, = sess.run(prob_tensor, {input_node: [image_tensor] })
    
print(predictions)

### View the results
The result of running the image tensor through the model is a vector of probabilities of the image belonging to each of the classes. To make the result more readable you will map it back to the labels.

In [ ]:
# Print the highest probability label
highest_probability_index = np.argmax(predictions)
print('Classified as: ' + labels[highest_probability_index])
print()

# Or you can print out all of the results mapping labels to probabilities.
label_index = 0
for p in predictions:
    truncated_probablity = np.float64(np.round(p,8))
    print (labels[label_index], truncated_probablity)
    label_index += 1

## Summary

In this lab, you exported the trained model and then used it in Python script for inference - a.k.a predictions. As you have seen, you can embedded your model in an arbitrary application. TensorFlow APIs are available in several languages besides Python, including JavaScript, C++, Java, Go, and Swift. In addition, you can use TensorFlow Serving and Azure Machine Learning service for model deployment in production.